
# Dashboard — Filters & Plots (Matplotlib)

Reads the **current package** chosen in `Start.ipynb`.  
Includes a tiny synthetic **sample** so you can see the UI immediately.


In [ ]:

    # Auto-select the sample so the app runs immediately (works both in Pyodide and CPython)
import os
os.makedirs("data_pkgs", exist_ok=True)
with open("data_pkgs/CURRENT.txt","w") as f:
    f.write("sample_local")


In [ ]:
from IPython.display import HTML, display
display(HTML("""
<style>
.jp-Cell-inputWrapper, .jp-InputPrompt, .jp-OutputPrompt { display: none !important; }
.jp-Toolbar, .jp-NotebookPanel-toolbar, #jp-top-panel { display: none !important; }
.jp-SideBar, .jp-LeftArea, .jp-RightArea { display: none !important; }
.jp-main-content-panel, .jp-Notebook { width: 100% !important; height: 100% !important; }
.jp-NotebookPanel { height: calc(100vh - 0px) !important; }
</style>
"""))


In [ ]:

# Install widgets (once per browser session)
import micropip
await micropip.install("ipywidgets==8.1.1")


In [ ]:

# Ensure a sample dataset locally and resolve current package
import os, numpy as np, pandas as pd

def ensure_sample(folder: str):
    os.makedirs(folder, exist_ok=True)
    meta_p = os.path.join(folder, "meta.csv")
    tuning_p = os.path.join(folder, "tuning_curves.npz")
    psth_p = os.path.join(folder, "psth.npz")

    if not (os.path.isfile(meta_p) and os.path.isfile(tuning_p) and os.path.isfile(psth_p)):
        # Create a tiny synthetic sample
        meta = pd.DataFrame({
            "unit_id": [f"U{i:03d}" for i in range(1, 41)],
            "layer":   np.random.choice(["SG","G","IG"], size=40),
            "osi":     np.round(np.random.rand(40), 3),
            "sel_index": np.round(0.4 + 0.6*np.random.rand(40), 3),
        })
        meta.to_csv(meta_p, index=False)

        angles = np.arange(0, 360, 10, dtype=np.float32)
        t = np.linspace(-1, 2, 1200, dtype=np.float32)
        tuning = {"angles": angles}
        psth = {"t": t}
        for uid in meta["unit_id"]:
            center = np.random.choice(angles)
            kappa  = np.random.uniform(2.5, 7.0)
            y = np.exp(np.cos(np.deg2rad(angles - center))*kappa).astype(np.float32)
            y = (y / y.max()) * np.random.uniform(0.8, 1.2)
            tuning[uid] = y

            base = 5 + 1.0*np.random.randn()
            r = base + 10*np.exp(-0.5*((t-0.5)/0.15)**2) + 1.5*np.random.randn(t.size)
            psth[uid] = np.clip(r, 0, None).astype(np.float32)

        np.savez(tuning_p, **tuning)
        np.savez(psth_p, **psth)

def current_package_path() -> str:
    DATA_ROOT = "data_pkgs"
    CURRENT_FILE = os.path.join(DATA_ROOT, "CURRENT.txt")
    if not os.path.isfile(CURRENT_FILE):
        # fallback: assume sample; create it under content/data or data
        for c in ["content/data", "data"]:
            try:
                ensure_sample(c)
                return c
            except Exception:
                pass
        ensure_sample("content/data")
        return "content/data"
    with open(CURRENT_FILE, "r") as f:
        pid = f.read().strip()
    if pid == "sample_local":
        for c in ["content/data", "data"]:
            if os.path.isdir(c):
                ensure_sample(c)
                return c
        ensure_sample("content/data")
        return "content/data"
    p = os.path.join("data_pkgs", pid)
    if not os.path.isdir(p):
        raise RuntimeError("Package folder missing. Re-select in Start.ipynb.")
    return p

PKG = current_package_path()
print("Using package:", PKG)


In [ ]:

# Load data
import numpy as np, pandas as pd, os
meta = pd.read_csv(os.path.join(PKG, "meta.csv"))
tun  = np.load(os.path.join(PKG, "tuning_curves.npz"))
psth = np.load(os.path.join(PKG, "psth.npz"))
angles = tun["angles"]
time   = psth["t"]
print(f"Units: {len(meta)} | angles: {len(angles)} | time points: {len(time)}")


In [ ]:

# UI: filters + plot selector
import ipywidgets as W
import matplotlib.pyplot as plt

layer_w = W.SelectMultiple(options=sorted(meta["layer"].unique()), description="Layer",
                           value=tuple(sorted(meta["layer"].unique())))
osi_w   = W.FloatRangeSlider(min=0, max=1, step=0.01, value=[0.2, 0.9],
                             description="OSI range", continuous_update=False)
sel_w   = W.FloatRangeSlider(min=0, max=1.2, step=0.01, value=[0.5, 1.1],
                             description="Sel range", continuous_update=False)
maxu_w  = W.IntSlider(min=1, max=20, step=1, value=5, description="#Units")
plot_type = W.ToggleButtons(options=["Tuning", "PSTH"], value="Tuning", description="Plot")

ui = W.VBox([layer_w, osi_w, sel_w, maxu_w, plot_type])
ui


In [ ]:

# Filtering + plotting
import matplotlib.pyplot as plt
import pandas as pd
import numpy as np
import ipywidgets as W

def apply_filter():
    q = meta[meta["layer"].isin(layer_w.value)]
    lo, hi = osi_w.value; q = q[(q["osi"] >= lo) & (q["osi"] <= hi)]
    lo2, hi2 = sel_w.value; q = q[(q["sel_index"] >= lo2) & (q["sel_index"] <= hi2)]
    return q.head(maxu_w.value)

out = W.Output()

def render():
    q = apply_filter()
    with out:
        out.clear_output(wait=True)
        display(q)
        if len(q)==0:
            print("No units match current filters."); return
        if plot_type.value == "Tuning":
            plt.figure(figsize=(6,3))
            for uid in q["unit_id"]:
                y = tun[uid]
                plt.plot(angles, y, alpha=0.9, label=uid)
            plt.title(f"Tuning curves (n={len(q)})")
            plt.xlabel("Angle (deg)"); plt.ylabel("Resp (a.u.)")
            plt.tight_layout(); plt.show()
        else:
            plt.figure(figsize=(6,3))
            for uid in q["unit_id"]:
                y = psth[uid]
                plt.plot(time, y, alpha=0.9, label=uid)
            plt.title(f"PSTH (n={len(q)})")
            plt.xlabel("Time (s)"); plt.ylabel("Rate (Hz)")
            plt.tight_layout(); plt.show()

for w in (layer_w, osi_w, sel_w, maxu_w, plot_type):
    w.observe(lambda _: render(), names="value")

render()
W.HBox([ui, out])



### Notes
- Replace the synthetic sample by adding real **packages** in `Start.ipynb` and downloading them.
- Keep real files named **`meta.csv`**, **`tuning_curves.npz`** (with key `angles` + per-unit arrays), and **`psth.npz`** (with key `t` + per-unit arrays).
- For more pages, create new notebooks (e.g., `Rasters.ipynb`) and reuse the `current_package_path()` helper.
